# Empirical Example

The purpose of this notebook is to demonstrate `ipcoal` simulations on a topology inferred from empirical data. We provide recommendations for how to scale units from a time-calibrated phylogeny to use in coalescent simulations, and how to incorporate biological information about species, such as generation times and population sizes, to perform more realistic simulations. 

Simulating coalescent genealogies and sequences on a parameterized species tree model using `ipcoal` can provide a null expectation for the amount of discordance that you expect to observe across different nodes of a species tree, and can even be used as a posterior predictive tool for phylogenetic analyses. 

In [1]:
import numpy as np
import pandas as pd
import ipcoal
import toytree
import toyplot
colormap = toyplot.color.brewer.map("BlueRed", reverse=True)

### Mammal phylogeny data set

In this example we use published data for mammals. We will use a time-calibrated MCC phylogeny by [Upham et al. (2009)](https://doi.org/10.1371/journal.pbio.3000494) as a species tree hypothesis; we will use species geographic areas from the [PanTHERIA database](https://doi.org/10.1890/08-1494.1) as a proxy for effective population sizes; and we will use generation time estimates from the [Pacifici et al. (2014)](https://doi.org/10.5061/dryad.gd0m3) data set, which imputes a lot of missing data from pantheria by using mean values among close relatives. 

In [2]:
# load the phylogenetic data (big tree, takes a few seconds)
TREE_URL = (
    "https://github.com/eaton-lab/ipcoal/blob/master/"
    "notebooks/mammal_dat/MamPhy_fullPosterior_BDvr_DNAonly"
    "_4098sp_topoFree_NDexp_MCC_v2_target.tre?raw=true"
)
tree = toytree.tree(TREE_URL, tree_format=10)
print(tree.ntips, "tips in the Upham mammal tree")

4100 tips in the Upham mammal tree


In [3]:
# load the mammal biological data (e.g., geo range)
PANTH_URL = (
    "https://github.com/eaton-lab/ipcoal/blob/master/"
    "notebooks/mammal_dat/PanTHERIA_1-0_WR05_Aug2008.txt?raw=true"
)
panthdf = pd.read_csv(PANTH_URL, sep='\t')
print(panthdf.shape[0], "taxa in PanTHERIA database")

5416 taxa in PanTHERIA database


In [4]:
# load the generation time data
GT_URL = (
    "https://github.com/eaton-lab/ipcoal/blob/master/"
    "notebooks/mammal_dat/5734-SP-2-Editor.csv?raw=true"
)
gentimedf = pd.read_csv(GT_URL)
print(gentimedf.shape[0], "taxa in Pacifici gentime database")

5427 taxa in Pacifici gentime database


### Filtering and selecting taxa

We will first trim the data down to include only taxa that are shared among all three data sources and for which there is no missing biological data. This reduces the data set to 3121 taxa. The distribution of geographic range areas is in units of kilometers$^2$ (`geogrange`) and generation times is in units of years (`gentime`).

In [5]:
# subselect species names and geo range columns from pantheria
sppdata = panthdf.loc[:, ['MSW05_Binomial', '26-1_GR_Area_km2']]

# rename sppdata columns
sppdata.columns = ["species", "georange"]

In [6]:
# make column to record tree tip label names
sppdata["treename"] = np.nan

# dict map: {gen}_{spp} to {gen}_{spp}_{fam}_{order}
tipdict = {i.rsplit("_", 2)[0]: i for i in tree.get_tip_labels()}

# record whether species in pantheria is in the tree tip labels
for idx in sppdata.index:
    
    # match data names to tree names which have underscores
    name = sppdata.species[idx]
    name_ = name.replace(" ", "_")
    
    # record treename if it is in the database
    if name_ in tipdict:
        sppdata.loc[idx, "treename"] = tipdict[name_]

In [7]:
# add gentime values to all species matching to names in Pacifici data set
sppdata["gentime"] = np.nan
for idx in gentimedf.index:
    
    # get generation time in units of years
    species, gent = gentimedf.loc[idx, ["Scientific_name", "GenerationLength_d"]] 
    mask = sppdata.species == species
    sppdata.loc[mask.values, "gentime"] = gent / 365.

In [8]:
# set missing data (-999) to NaN
sppdata[sppdata == -999.000] = np.nan

# remove rows where either georange or gentime is missing
mask = sppdata.georange.notna() & sppdata.gentime.notna() & sppdata.treename.notna()
sppdata = sppdata.loc[mask, :]

# reorder and reset index for dropped rows
sppdata.sort_values(by="species", inplace=True)
sppdata.reset_index(drop=True, inplace=True)

# show first ten sorted rows
sppdata.head(10)

,species,georange,treename,gentime
0,Abeomelomys sevia,53261.73,Abeomelomys_sevia_MURIDA...,1.710684
1,Abrocoma bennettii,54615.98,Abrocoma_bennettii_ABROC...,2.829928
2,Abrocoma boliviensis,5773.97,Abrocoma_boliviensis_ABR...,2.829928
3,Abrocoma cinerea,381391.02,Abrocoma_cinerea_ABROCOM...,2.829928
4,Abrothrix andinus,722551.83,Abrothrix_andinus_CRICET...,1.614762
5,Abrothrix hershkovitzi,1775.72,Abrothrix_hershkovitzi_C...,1.614762
6,Abrothrix illuteus,35359.55,Abrothrix_illuteus_CRICE...,1.614762
7,Abrothrix jelskii,506394.71,Abrothrix_jelskii_CRICET...,1.614762
8,Abrothrix lanosus,43016.67,Abrothrix_lanosus_CRICET...,1.614762
9,Abrothrix longipilis,423823.71,Abrothrix_longipilis_CRI...,1.614762


#### Filter the tree to include only taxa in the data table


In [9]:
# find names in tree but not in data table
names_in_data = set(sppdata.treename)
names_in_tree = set(tree.get_tip_labels())
names_to_remove = names_in_tree.difference(names_in_data)

In [10]:
# drop the tips from the tree not in data table
ftree = tree.drop_tips(names_to_remove)
print(len(ftree), "tips in filtered tree (ftree)")

3121 tips in filtered tree (ftree)


#### Convert geographic ranges to Ne values
Here we generate a range of Ne values within a selected range that are scaled by the variation in geographic range area sizes among taxa. The distribution is plotted as a histrogram on a y-axis log scale. Many taxa have small Ne, few have very large Ne. 

In [11]:
# transform georange into Ne values within selected range
max_Ne = 1000000
min_Ne = 1000

# set Ne values in range scaled by geographic ranges
Ne = max_Ne * (sppdata.georange / sppdata.georange.max())
Ne = [max(min_Ne, i) for i in Ne]
sppdata["Ne"] = np.array(Ne, dtype=int)

# show 10 random samples
sppdata.sample(10)

,species,georange,treename,gentime,Ne
1090,Heteromys desmarestianus,479714.68,Heteromys_desmarestianus...,2.393204,7610
1498,Melasmothrix naso,2059.07,Melasmothrix_naso_MURIDA...,1.710684,1000
2234,Phyllomys brasiliensis,148527.87,Phyllomys_brasiliensis_E...,2.024684,2356
1244,Lasiurus cinereus,21759652.98,Lasiurus_cinereus_VESPER...,5.621003,345203
1429,Marmosa mexicana,905454.61,Marmosa_mexicana_DIDELPH...,1.557911,14364
3097,Vulpes ferrilata,2339623.06,Vulpes_ferrilata_CANIDAE...,3.910419,37116
698,Dasymys rufulus,861651.19,Dasymys_rufulus_MURIDAE_...,1.710684,13669
2277,Platalina genovensium,672990.49,Platalina_genovensium_PH...,5.655552,10676
2122,Pelomys fallax,5222579.33,Pelomys_fallax_MURIDAE_R...,1.752416,82852
1292,Lepus californicus,4402904.48,Lepus_californicus_LEPOR...,3.360291,69849


In [12]:
# plot a histogram of Ne values
a, b = np.histogram(sppdata.Ne, bins=25)
toyplot.bars((a, b), height=300, yscale="log", ylabel="bin count", xlabel="Ne");

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t19866ef706854e3cbec9e915c305598b" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600 300.0" width="600px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 500000 1000000 Ne 0 10 0 10 1 10 2 10 3 10 4 bin count

#### Set Ne and g values for tip and ancestral nodes on the tree object

*ipcoal* can accept different Ne and g values to use in simulations, and the easiest way to set variable values across different parts of the tree is to map the values to the tree object that *ipcoal* accepts as an argument. We only have estimates of Ne and g for species that are alive today, but it would be useful to also includes estimates for ancestral nodes in the species tree. Here we use a simple ancestral state reconstruction based on Brownian motion to infer states for ancestral nodes. 

In [13]:
# make a copy of the filtered tree
tree_ng = ftree.copy() 

# dictionaries mapping names to values
dict_ne = {sppdata.treename[i]: sppdata.Ne[i] for i in range(sppdata.shape[0])}
dict_gt = {sppdata.treename[i]: sppdata.gentime[i] for i in range(sppdata.shape[0])}

# set values on nodes of the tree for all species (tips)
tree_ng = tree_ng.set_node_values("Ne", dict_ne)
tree_ng = tree_ng.set_node_values("g", dict_gt)

# estimate and set values on ancestral nodes as well.
tree_ng = tree_ng.pcm.ancestral_state_reconstruction("g")
tree_ng = tree_ng.pcm.ancestral_state_reconstruction("Ne")

### Plot tree with Ne and g values 

Let's plot just a subset of taxa to start, since it will be much easier to visualize than trying to examine the entire tree. Here we select only the taxa in the genus *Mustela*. The tree plot shows variation in Ne using the thickness of edges, and generation times are shows by the color of nodes, blue to red, representing shorter to longer times. The `ts='p'` drawing option automatically pulls the Ne information from the nodes of the tree to draw the edge thickness. 

In [14]:
# make a tree copy
atree = tree_ng.copy()

# get ancestor of all tips that have 'Mustela' in their name
mrca_node_idx = atree.get_mrca_idx_from_tip_labels(wildcard="Mustela_")

# get the TreeNode object of this subtree
node = atree.get_feature_dict("idx")[mrca_node_idx]

# create as a new Toytree
subtree = toytree.tree(node)

# scale the tree height from millions of year to years
subtree = subtree.mod.node_scale_root_height(subtree.treenode.height * 1e6)

In [15]:
subtree.draw(
    ts='p', 
    edge_type='p',
    node_sizes=10,
    node_labels=False,
    node_colors=[
        colormap.colors(i, 0.1, 10) for i in subtree.get_node_values('g', 1, 1)
    ],
    width=400, 
    height=600,
);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="t6459a84d236a4ec8b92496cd7b5c15fe" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 600.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA 0 1615584 3231169 4846753 6462338 8077922

### Convert edge lengths from time to generations

Time in years is converted to units of generations by dividing by each edge length by the generation time for that edge, recorded as ngenerations/year. When this is done the crown root age of the *Mustela* tree is now at 2.4M generations from the furthest tip in the tree. This tree object (ttree) now contains information in its Ne values mapped to nodes and in its edge lengths to fully represent the data on population sizes and generation time differences among species and their ancestors. **This is the tree we will use for our ipcoal simulations.**


In [16]:
# divide the edge lengths (in abosolute time) by the generation time
ttree = subtree.set_node_values(
    "dist",
    {i.name: i.dist / i.g for i in subtree.get_feature_dict()}
)

In [17]:
ttree.draw(
    ts='p',
    edge_type='p',
    tip_labels_align=True,
    tip_labels=[i.rsplit("_", 2)[0] for i in ttree.get_tip_labels()],
    node_labels=False,
    node_sizes=0,
    width=400, 
    height=400,
);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="t38b9625bbdea4f3cb4ecf17fcb6d388d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 400.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Mustela_eversmanii Mustela_putorius Mustela_lutreola Mustela_sibirica Mustela_itatsi Mustela_altaica Mustela_nivalis Mustela_erminea Mustela_kathiah Mustela_nudipes Mustela_strigidorsa Mustela_felipei Mustela_africana Mustela_frenata Neovison_vison 0 805778 1611557 2417335

### Examine the distribution of genealogy lengths

Here we will finally start to simulate data. The tree topology, edge lengths, and Ne values will all be inherited from the tree object by *ipcoal* to setup the coalescent simulation parameters. We leave the rest of the parameters at their default values (e.g., mutation rate and recombination rate). Let's first simulate a large chromosome and examine the lengths of genealogical blocks across the genome. If they are very small then we may have to worry that given the scale of our data set recombination is likely to have occurred within the length of a single locus. 

In [18]:
# initialize the ipcoal model object
mod = ipcoal.Model(ttree, seed=333)

# simulate 1 chromosome 1Mb is length
mod.sim_trees(nloci=1, nsites=1e6)

# print the average length of a non-recombined region
print(mod.df.groupby("locus").apply(len).mean())

7605.0


In [19]:
canvas, axes, mark = toyplot.bars(
    np.histogram(mod.df.nbps, 50, range=[0, 1000]),
    width=300, 
    height=300,
    xlabel="gene tree length (bp)",
    ylabel="bin count",
    label="The size of non-recombined genomic blocks",
)

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t02fb0f9940cf42e99db535a4d2069df7" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 300.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 500 1000 gene tree length (bp) 0 500 1000 bin count The size of non-recombined genomic blocks

### Simulate unlinked loci (e.g., UCE)

The figure above should give us some concern. The average length of a non-recombined region is far less than 500bp, and thus a typical phylogenetic marker, like a UCE, may actually represent multiple distinct genealogies (i.e., concatenation). If we think recombination may be suppressed in our marker region (e.g., a UCE) then you could lower the per-site per-generation recombination rate (and similarly you may want to lower the mutation rate). 

In [25]:
# initialize the ipcoal model object
mod = ipcoal.Model(ttree, seed=333, recomb=1e-10, mut=1e-9)

# simulate many 1000bp loci
mod.sim_loci(nloci=50, nsites=1000)

# show the dataframe of genealogy results
mod.df.head()

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,407,407,9,0,((Mustela_kathiah_MUSTEL...
1,0,407,1000,593,9,1,((Mustela_kathiah_MUSTEL...
2,1,0,41,41,1,0,((Neovison_vison_MUSTELI...
3,1,41,309,268,6,1,((Neovison_vison_MUSTELI...
4,1,309,1000,691,18,2,((Neovison_vison_MUSTELI...


In [26]:
# a dictionary of tree drawing styles that we will use
kwargs = {
    "tip_labels_align": True,
    "tip_labels_style": {"font-size": "9px"},
}

In [27]:
# randomly sample 8 genealogy indices
rand = mod.df.sample(8).index

# draw linked genealogies
toytree.mtree(mod.df.genealogy[rand]).draw_tree_grid(ncols=4, nrows=2, **kwargs);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t0436848fc96e42a594d8df6ae5657c66" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 500.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA <text style="fill:rgb(14.9%,14.9%,14.9%);fi

### Examine expected variation in inferred gene trees
The substitution process is modeled on top of the simulated genealogies to produce sequence data, and ML gene trees are inferred from the resulting sequence data. It can be useful to compare inferred trees to the true genealogies to find which parts of the species tree are most difficult to infer, i.e., which splits are most affected not only by genealogical variation but also by gene tree estimation error caused by low information content or homoplasy. 

In [28]:
# infer a gene tree for each LOCUS
mod.infer_gene_trees()

In [30]:
# show resulting true genealogies and inferred gene trees
mod.df.head()

,locus,start,end,nbps,nsnps,tidx,genealogy,inferred_tree
0,0,0,407,407,9,0,((Mustela_kathiah_MUSTEL...,((((((((Mustela_lutreola...
1,0,407,1000,593,9,1,((Mustela_kathiah_MUSTEL...,((((((((Mustela_lutreola...
2,1,0,41,41,1,0,((Neovison_vison_MUSTELI...,(Mustela_felipei_MUSTELI...
3,1,41,309,268,6,1,((Neovison_vison_MUSTELI...,(Mustela_felipei_MUSTELI...
4,1,309,1000,691,18,2,((Neovison_vison_MUSTELI...,(Mustela_felipei_MUSTELI...


In [61]:
# draw inferred unrooted gene trees
rand = mod.df.sample(8).index
toytree.mtree(mod.df.inferred_tree[rand]).draw_tree_grid(ncols=4, nrows=2, **kwargs);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t62289ac97d88424ea36776b84d32d22e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 500.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA <path d="M 243.3333333333

In [60]:
# get consensus of all inferred ML trees
ctree = toytree.mtree(mod.df.inferred_tree).get_consensus_tree()

# draw the consensus tree
ctree.draw(
    ts='n', 
    edge_type='p', 
    node_labels="idx", #support", 
    use_edge_lengths=False,
);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="t4e170ea7b7c54a72ade00b50e65c9392" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 450.0 275.0" width="450.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA 15 16 17 18 19 20 21 22 23 24 25 26 27

### Simulate on a larger clade  
  
Let's scale up to Mustelidae, the whole family containing weasels, ferrets, otters, etc.

In [44]:
# make a tree copy
atree = tree_ng.copy()

# get ancestor of all tips that have 'Mustela' in their name
mrca_node_idx = atree.get_mrca_idx_from_tip_labels(wildcard="MUSTELIDAE")

# make new toytree from selected node
node = atree.get_feature_dict("idx")[mrca_node_idx]
subtree = toytree.tree(node)

# scale the tree height from millions of year to years
subtree = subtree.mod.node_scale_root_height(subtree.treenode.height * 1e6)

In [46]:
canvas, axes = subtree.draw(
    ts='p', 
    edge_type='p',
    node_sizes=10,
    node_labels=False,
    node_colors=[
        colormap.colors(i, 0.1, 10) for i in subtree.get_node_values('g', 1, 1)
    ],
    width=800, 
    height=600,
);

# set y axis ticks at 2My intervals and fit long tip names
axes.y.ticks.locator = toyplot.locator.Explicit(
    range(0, int(2e7), int(2e6)),
    [str(int(i / 1e6)) for i in range(0, int(2e7), int(2e6))],
)
axes.y.domain.min = -20e6

<svg class="toyplot-canvas-Canvas" height="600.0px" id="te23e8b4b4c434e93a9c6977c24c4a12c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 600.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Aonyx_cinerea_MUSTELIDAE_CARNIVORA Lutrogale_perspicillata_MUSTELIDAE_CARNIVORA Aonyx_capensis_MUSTELIDAE_CARNIVORA Lutra_lutra_MUSTELIDAE_CARNIVORA Lutra_sumatrana_MUSTELIDAE_CARNIVORA Lontra_provocax_MUSTELIDAE_CARNIVORA Lontra_longicaudis_MUSTELIDAE_CARNIVORA Lontra_canadensis_MUSTELIDAE_CARNIVORA Pteronura_brasiliensis_MUSTELIDAE_CARNIVORA Ictonyx_striatus_MUSTELIDAE_CARNIVORA Poecilogale_albinucha_MUSTELIDAE_CARNIVORA Ictonyx_libyca_MUSTELIDAE_CARNIVORA Vormela_peregusna_MUSTELIDAE_CARNIVORA Lyncodon_patagonicus_MUSTELIDAE_CARNIVORA Galictis_vittata_MUSTELIDAE_CARNIVORA Galictis_cuja_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Melogale_moschata_MUSTELIDAE_CARNIVORA Melogale_personata_MUSTELIDAE_CARNIVORA Martes_martes_MUSTELIDAE_CARNIVORA Martes_zibellina_MUSTELIDAE_CARNIVORA Martes_melampus_MUSTELIDAE_CARNIVORA Martes_americana_MUSTELIDAE_CARNIVORA Martes_foina_MUSTELIDAE_CARNIVORA Martes_flavigula_MUSTELIDAE_CARNIVORA Gulo_gulo_MUSTELIDAE_CARNIVORA Martes_pennanti_MUSTELIDAE_CARNIVORA Eira_barbara_MUSTELIDAE_CARNIVORA Mellivora_capensis_MUSTELIDAE_CARNIVORA Arctonyx_collaris_MUSTELIDAE_CARNIVORA Taxidea_taxus_MUSTELIDAE_CARNIVORA 0 2 4 6 8 10 12 14 16 18

In [47]:
# convert time to generations
ttree = subtree.set_node_values(
    "dist",
    {i.name: i.dist / i.g for i in subtree.get_feature_dict()}
)

In [48]:
ttree.draw(
    ts='p',
    edge_type='p',
    tip_labels_align=True,
    node_labels=False,
    node_sizes=0,
    width=800, 
    height=600,
);

<svg class="toyplot-canvas-Canvas" height="600.0px" id="tf570d49e890f497ab3f829261385bde5" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 600.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Aonyx_cinerea_MUSTELIDAE_CARNIVORA Lutrogale_perspicillata_MUSTELIDAE_CARNIVORA Aonyx_capensis_MUSTELIDAE_CARNIVORA Lutra_lutra_MUSTELIDAE_CARNIVORA Lutra_sumatrana_MUSTELIDAE_CARNIVORA Lontra_provocax_MUSTELIDAE_CARNIVORA Lontra_longicaudis_MUSTELIDAE_CARNIVORA Lontra_canadensis_MUSTELIDAE_CARNIVORA Pteronura_brasiliensis_MUSTELIDAE_CARNIVORA Ictonyx_striatus_MUSTELIDAE_CARNIVORA Poecilogale_albinucha_MUSTELIDAE_CARNIVORA Ictonyx_libyca_MUSTELIDAE_CARNIVORA Vormela_peregusna_MUSTELIDAE_CARNIVORA Lyncodon_patagonicus_MUSTELIDAE_CARNIVORA Galictis_vittata_MUSTELIDAE_CARNIVORA Galictis_cuja_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Melogale_moschata_MUSTELIDAE_CARNIVORA Melogale_personata_MUSTELIDAE_CARNIVORA Martes_martes_MUSTELIDAE_CARNIVORA Martes_zibellina_MUSTELIDAE_CARNIVORA Martes_melampus_MUSTELIDAE_CARNIVORA Martes_americana_MUSTELIDAE_CARNIVORA Martes_foina_MUSTELIDAE_CARNIVORA Martes_flavigula_MUSTELIDAE_CARNIVORA Gulo_gulo_MUSTELIDAE_CARNIVORA Martes_pennanti_MUSTELIDAE_CARNIVORA Eira_barbara_MUSTELIDAE_CARNIVORA Mellivora_capensis_MUSTELIDAE_CARNIVORA Arctonyx_collaris_MUSTELIDAE_CARNIVORA Taxidea_taxus_MUSTELIDAE_CARNIVORA 0 985752 1971503 2957255 3943007 4928759